In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
%matplotlib inline

from PIL import Image
import os 
import glob
import random

random.seed(42)


In [10]:
# Restart

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox

# Load the CSV file
df = pd.read_csv("data.csv")
# Set your base path where images are stored.
image_base_path = "BMI/Data/Images"  # update this to your actual image directory

# Load the CSV file
df = pd.read_csv("data.csv")

# Create a new column 'file_path' with the full path for each image.
df['file_path'] = df['name'].apply(lambda x: os.path.join(image_base_path, x))
# df['bmi_boxcox'], fitted_lambda = boxcox(df['bmi_adjusted'])
# Keep only the rows where the file exists.
df = df[df['file_path'].apply(os.path.exists)]
df['sex'] = df['gender'].map({'Male': 0, 'Female': 1})



In [14]:
# Split into training and test sets based on the 'is_training' flag
df_train_full = df[df['is_training'] == 1]
df_test = df[df['is_training'] == 0]

# Further split the full training set into train and validation sets 
df_train, df_valid = train_test_split(df_train_full, test_size=0.1, random_state=42)

In [16]:
import os



def build_img_list(df, image_base_path):
    """
    Returns a list of tuples: (full_image_path, bmi, sex)
    Assumes df contains columns: 'name', 'bmi', and 'sex'
    """
    return [
        (os.path.join(image_base_path, row['name']), row['bmi'], row['sex'])
        for _, row in df.iterrows()
    ]

train_img_lst = build_img_list(df_train, image_base_path)
valid_img_lst = build_img_list(df_valid, image_base_path)
test_img_lst  = build_img_list(df_test, image_base_path)


In [18]:
import torchvision.transforms as transforms

train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(degrees=20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.2010]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33))
])


valid_transforms = transforms.Compose([
    transforms.Resize((256, 256)),              # Resize to 256x256
    transforms.CenterCrop((224, 224)),            # Center crop to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.2010])
])

# Try: more advanced data augmentation

In [20]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class CustomImageDataset(Dataset):
    def __init__(self, img_list, transform=None):
        """
        img_list: List of tuples (image_path, bmi, sex)
        transform: Image transformations to apply
        """
        self.img_list = img_list  
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img_path, bmi, sex = self.img_list[idx]
        # Open the image and convert to RGB
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        
        # Convert bmi and sex to tensors (adjust types as needed)
        bmi = torch.tensor(bmi, dtype=torch.float32)
        sex = torch.tensor(sex, dtype=torch.float32)
        
        # Return the image and its corresponding features/labels.
        return image, bmi, sex


In [22]:
df

,Unnamed: 0,bmi,gender,is_training,name,file_path,sex
0,0,34.207396,Male,1,img_0.bmp,BMI/Data/Images\img_0.bmp,0
1,1,26.453720,Male,1,img_1.bmp,BMI/Data/Images\img_1.bmp,0
2,2,34.967561,Female,1,img_2.bmp,BMI/Data/Images\img_2.bmp,1
3,3,22.044766,Female,1,img_3.bmp,BMI/Data/Images\img_3.bmp,1
6,6,25.845588,Female,1,img_6.bmp,BMI/Data/Images\img_6.bmp,1
...,...,...,...,...,...,...,...
4201,4201,34.078947,Male,0,img_4201.bmp,BMI/Data/Images\img_4201.bmp,0
4202,4202,34.564776,Female,0,img_4202.bmp,BMI/Data/Images\img_4202.bmp,1
4203,4203,27.432362,Female,0,img_4203.bmp,BMI/Data/Images\img_4203.bmp,1
4204,4204,40.492800,Male,0,img_4204.bmp,BMI/Data/Images\img_4204.bmp,0


In [24]:
# Create your datasets using your lists and appropriate transforms
train_data = CustomImageDataset(train_img_lst, transform=train_transforms)
valid_data = CustomImageDataset(valid_img_lst, transform=valid_transforms)
test_data = CustomImageDataset(test_img_lst, transform=valid_transforms)

In [65]:
batch_size = 16

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [67]:
for batch, (X, y, sex) in enumerate(train_loader):
    if torch.isnan(X).any():
        print(f"NaN detected in images at batch {batch}")
    if torch.isnan(y).any():
        print(f"NaN detected in BMI targets at batch {batch}")
    if torch.isnan(sex).any():
        print(f"NaN detected in sex feature at batch {batch}")
    



In [68]:
import torch
import torch.nn as nn
import torchvision

class ResNet50(nn.Module):
    def __init__(self, load_weights=True, freeze_hidden_layers=False):
        super(ResNet50, self).__init__()

        # Load the pretrained ResNet50 model.
        self.base_model = torchvision.models.resnet50(pretrained=load_weights)
        
        # Get the number of features from the original fc layer.
        num_ftrs = self.base_model.fc.in_features
        
        # Remove the original fc layer.
        self.base_model.fc = nn.Identity()
        
        # Create a new fc layer that accepts both the image features and the extra sex feature.
        # We add one extra input feature to account for sex.
        self.fc_reg = nn.Linear(num_ftrs + 1, 1)
        
        if freeze_hidden_layers:
            for name, param in self.base_model.named_parameters():
                param.requires_grad = False
            # Optionally, allow gradients for the new fc_reg layer
            for param in self.fc_reg.parameters():
                param.requires_grad = True

    def forward(self, x, sex):
        # Get features from the base model.
        features = self.base_model(x)
        # Ensure the sex feature has shape [batch_size, 1]
        if len(sex.shape) == 1:
            sex = sex.unsqueeze(1)
        # Concatenate the image features with the sex feature.
        combined = torch.cat((features, sex), dim=1)
        # Produce the final regression output.
        output = self.fc_reg(combined)
        return output


In [69]:
import torch
import torchvision
import torch.nn as nn

In [70]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
print(f'Device Currently Using: {torch.cuda.get_device_name(torch.cuda.current_device())}')

Using cuda device
Device Currently Using: NVIDIA GeForce RTX 3060


In [71]:
def train(dataloader, model, loss_function, optimizer):
    # Set model to training mode.
    model.train()
    size = len(dataloader.dataset)

    for batch, (X, y, sex) in enumerate(dataloader):
        # Move inputs and labels to device
        X, y, sex = X.to(device), y.to(device), sex.to(device)

        # Forward pass: pass both image and sex feature to your model.
        pred = model(X, sex)         
        # loss = loss_function(pred, y) 
        loss = loss_function(pred.squeeze(1), y)

        optimizer.zero_grad()  
        loss.backward()        
        optimizer.step()     

        if batch % 32 == 0:
            loss_val, current = loss.item(), batch * len(X)
            print(f"loss: {loss_val:>7f}  [{current:>5d}/{size:>5d}]")


In [72]:
def valid(dataloader, model, loss_function):
    model.eval()
    total_loss = 0.0
    total_abs_error = 0.0
    total_samples = 0
    total_mae_loss = 0.0
    with torch.no_grad(): 
        for X, y, sex in dataloader:
            # Move inputs and labels (and extra feature) to device.
            X, y, sex = X.to(device), y.to(device), sex.to(device)
            
            # Forward pass with the extra feature
            pred = model(X, sex)
            
            # Compute loss
            
            # loss = loss_func(pred, y)
            loss = loss_function(pred.squeeze(1), y)
            mae_loss = nn.L1Loss()(pred.squeeze(1), y)
            batch_size = X.size(0)
            total_loss += loss.item() * batch_size
            total_mae_loss += mae_loss.item() * batch_size
            # Compute absolute error for regression (MAE)
            total_abs_error += torch.abs(pred - y).sum().item()
            
            total_samples += batch_size
            


    avg_loss = total_loss / total_samples
    avg_mae_loss = total_mae_loss / total_samples
    avg_abs_error = total_abs_error / total_samples
    print(f"Validation Error: \n MSE Loss: {avg_loss:>8f} \n MAE Loss: {avg_mae_loss:>8f}\n")
    return avg_loss, avg_mae_loss


In [73]:
def test(dataloader, model, loss_function):
    y_true, y_pred = [], []
    total_loss = 0.0
    total_samples = 0

    model.eval()

    with torch.no_grad(): 
        for X, y, sex in dataloader:
            # Move the image, BMI label, and sex feature to the device.
            X, y, sex = X.to(device), y.to(device), sex.to(device)

            # Forward pass: pass both the image and sex feature to your model.
            pred = model(X, sex)
            
            # Accumulate predictions and true values.
            y_pred.extend(pred.view(-1).detach().cpu().numpy())
            y_true.extend(y.view(-1).detach().cpu().numpy())

            # Multiply by batch size for an accurate total loss.
            batch_size = X.size(0)
            total_loss += loss_function(pred.squeeze(1), y)* batch_size
            total_samples += batch_size

    # Calculate the average loss over all samples.
    avg_loss = total_loss / total_samples

    # Calculate a regression metric (e.g., Mean Absolute Error)
    mae = sum(abs(p - t) for p, t in zip(y_pred, y_true)) / total_samples

    print(f"Test Error:\n Avg Loss: {avg_loss:>8f}\n Avg MAE: {mae:>8f}")
    return avg_loss, mae, y_true, y_pred


In [74]:
optimizers = ['Adam', 'SGD']
# learning_rates = np.logspace(-6, -1, 10)
learning_rates = np.logspace(-5, -4, 10)

# learning_rates = learning_rates[3:10]
print(learning_rates)

[1.00000000e-05 1.29154967e-05 1.66810054e-05 2.15443469e-05
 2.78255940e-05 3.59381366e-05 4.64158883e-05 5.99484250e-05
 7.74263683e-05 1.00000000e-04]


In [83]:
grid_hist = {}

In [102]:
for opt in optimizers:
    grid_hist[opt] = {}
    for lr_idx, lr in enumerate(learning_rates):
        if lr_idx >= 4:
            print('Skipping lr of', learning_rates[lr_idx])
            continue

        # Create a new model instance that expects both image and sex as inputs.
        model = ResNet50().to(device)  # Ensure model.forward accepts (X, sex)
        
        grid_hist[opt][lr_idx] = {}
        grid_hist[opt][lr_idx]['minLoss'] = 0
        print('==============Training with {} at lr = {}=============='.format(opt, lr))
        
        # Choose optimizer based on option.
        if opt == 'Adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
        elif opt == 'SGD':
            optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=1e-5)

        # For regression (e.g., BMI prediction), use Mean Squared Error Loss.
        criterion = nn.MSELoss()  
        max_epochs = 250
        current_min_loss = np.inf
        current_best_metric = np.inf  
        current_best_pcorr = 0.0
        earlyStop_cnt = 0
        for t in range(max_epochs):
            print(f"Epoch {t+1}\n-------------------------------")
            # The train function should unpack (X, y, sex) and call model(X, sex)
            train(train_loader, model, criterion, optimizer)
            # Similarly, valid should be modified to handle the extra feature.
            eval_loss, eval_metric = valid(valid_loader, model, criterion)

            # Save the best model based on the lowest loss.
            if eval_metric < current_min_loss:
                torch.save(model.state_dict(), 'best_lr_resnet50_{}_{}.pth'.format(opt, lr_idx))
                print('Best Saved!', eval_metric)
                current_min_loss = eval_metric
                earlyStop_cnt = 0
                
                from scipy.stats import pearsonr
                def evaluate_model(model, dataloader, device):
                    """
                    Evaluate the model on a dataloader and return the ground truth and predicted BMI values.
                    Assumes each batch from the dataloader returns a tuple: (X, y, sex)
                    """
                    model.eval()
                    all_preds = []
                    all_targets = []
                    with torch.no_grad():
                        for X, y, sex in dataloader:
                            X, y, sex = X.to(device), y.to(device), sex.to(device)
                            preds = model(X, sex)
                            # Squeeze predictions from [batch_size, 1] to [batch_size]
                            preds = preds.squeeze(1)
                            all_preds.extend(preds.cpu().numpy())
                            all_targets.extend(y.cpu().numpy())
                    return np.array(all_targets), np.array(all_preds)
                y_true, y_pred = evaluate_model(model, test_loader, device)
                pearson_corr, p_value = pearsonr(y_true, y_pred)
                if pearson_corr > current_best_pcorr:
                    torch.save(model.state_dict(), 'best_corr_resnet50_{}_{}.pth'.format(opt, lr_idx))
                    current_best_pcorr = pearson_corr
                    print(f"Current Best pcorr: {current_best_pcorr}")
                    
            else:
                earlyStop_cnt += 1
            
            # For regression, lower metric (e.g., MAE) is better.
            # if eval_metric < current_best_metric:
            #     print('Best Metric:', eval_metric)
            #     current_best_metric = eval_metric

            grid_hist[opt][lr_idx]['minLoss'] = current_min_loss
            grid_hist[opt][lr_idx]['bestMetric'] = current_best_metric

            if earlyStop_cnt >= 20:
                print('Early Stopped!')
                break
            print('Current no-progress epoch:', earlyStop_cnt, '\n')


C:\Users\chien\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\chien\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


==============Training with Adam at lr = 1e-05==============
Epoch 1
-------------------------------
loss: 968.330444  [    0/ 2889]
loss: 1343.946533  [  512/ 2889]
loss: 912.424316  [ 1024/ 2889]
loss: 1080.367188  [ 1536/ 2889]
loss: 750.888977  [ 2048/ 2889]
loss: 875.790527  [ 2560/ 2889]
Validation Error: 
 MSE Loss: 834.379977 
 MAE Loss: 27.685800

Best Saved! 27.685800314692322
Current Best pcorr: 0.24838215539889122
Current no-progress epoch: 0 

Epoch 2
-------------------------------
loss: 1033.884033  [    0/ 2889]
loss: 856.967163  [  512/ 2889]
loss: 764.287964  [ 1024/ 2889]
loss: 659.794800  [ 1536/ 2889]
loss: 720.170898  [ 2048/ 2889]
loss: 600.475220  [ 2560/ 2889]
Validation Error: 
 MSE Loss: 703.747428 
 MAE Loss: 25.255252

Best Saved! 25.2552521845262
Current Best pcorr: 0.25226267175532735
Current no-progress epoch: 0 

Epoch 3
-------------------------------
loss: 680.855957  [    0/ 2889]
loss: 623.159180  [  512/ 2889]
loss: 582.123047  [ 1024/ 2889]
loss: 